In [1]:
import pickle 
import pandas as pd 
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

42997


In [4]:
file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\New Feature Creation\input_list.npy', 'rb')
inputs = pickle.load(file)

file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\New Feature Creation\classes_list.npy', 'rb')
classes = pickle.load(file)
#seperate out classes from inputs
#print(f"{[len(x) for x  in inputs]}")
inputs_length = pd.Series(data =[len(x) for x  in inputs] )
# #removing nan from inputs and convert to float
# inputs_df = pd.DataFrame(inputs)
# inputs_df.fillna(value=-1,inplace = True)
# inputs = inputs_df.values.tolist()
# inputs = [[float(i) for i in j] for j in inputs]

In [5]:

#reading data from pickle
file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\New Feature Creation\input_list.npy', 'rb')
inputs = pickle.load(file)

file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\New Feature Creation\classes_list.npy', 'rb')
classes = pickle.load(file)

data = zip(inputs,classes)
new_inputs = []
new_classes = []
for i,j in data:
    if len(i) == 32:
        new_inputs.append(i)
        new_classes.append(j)
        continue
    if len(i) ==28:
        i_new = i + [-1,-1,-1,-1]
        new_inputs.append(i_new)
        new_classes.append(j+[8])
        print(new_classes[-1])
inputs = new_inputs
classes = new_classes
#removing nan from inputs and convert to float
inputs_df = pd.DataFrame(inputs)
inputs_df.fillna(value=-1,inplace = True)
inputs = inputs_df.values.tolist()
inputs = [[float(i) for i in j] for j in inputs]

[3.0, 7.0, 1.0, 2.0, 4.0, 6.0, 5.0, 8]
[6.0, 3.0, 7.0, 4.0, 5.0, 2.0, 1.0, 8]
[1.0, 6.0, 2.0, 5.0, 3.0, 4.0, 7.0, 8]
[1.0, 5.0, 2.0, 4.0, 6.0, 3.0, 7.0, 8]
[3.0, 2.0, 1.0, 6.0, 5.0, 4.0, 7.0, 8]
[2.0, 3.0, 6.0, 1.0, 4.0, 7.0, 5.0, 8]
[4.0, 7.0, 1.0, 2.0, 6.0, 5.0, 3.0, 8]
[7.0, 4.0, 5.0, 2.0, 3.0, 1.0, 6.0, 8]
[1.0, 3.0, 6.0, 4.0, 2.0, 7.0, 5.0, 8]
[3.0, 4.0, 6.0, 2.0, 5.0, 1.0, 7.0, 8]
[2.0, 5.0, 3.0, 7.0, 4.0, 6.0, 1.0, 8]
[5.0, 2.0, 1.0, 7.0, 3.0, 6.0, 4.0, 8]
[7.0, 1.0, 6.0, 3.0, 4.0, 5.0, 2.0, 8]
[7.0, 3.0, 2.0, 4.0, 6.0, 1.0, 5.0, 8]
[7.0, 1.0, 3.0, 2.0, 6.0, 5.0, 4.0, 8]
[4.0, 2.0, 7.0, 6.0, 3.0, 1.0, 5.0, 8]
[2.0, 6.0, 5.0, 7.0, 4.0, 3.0, 1.0, 8]
[2.0, 4.0, 7.0, 6.0, 5.0, 1.0, 3.0, 8]
[4.0, 2.0, 7.0, 3.0, 5.0, 1.0, 6.0, 8]
[6.0, 4.0, 3.0, 5.0, 1.0, 7.0, 2.0, 8]
[1.0, 5.0, 4.0, 2.0, 7.0, 6.0, 3.0, 8]
[5.0, 3.0, 2.0, 7.0, 4.0, 6.0, 1.0, 8]
[6.0, 4.0, 3.0, 2.0, 7.0, 5.0, 1.0, 8]
[7.0, 6.0, 5.0, 1.0, 4.0, 3.0, 2.0, 8]
[3.0, 7.0, 5.0, 6.0, 2.0, 1.0, 4.0, 8]
[7.0, 5.0, 4.0, 1.0, 3.0,

In [ ]:
#len(new_inputs)
len(inputs)

42997

In [7]:
inputs_length = pd.Series(data =[len(x) for x  in new_inputs] )
inputs_length.value_counts()

32    42997
dtype: int64

In [6]:
#these are fine
def train(net):
    BATCH_SIZE = 100
    EPOCHS = 1000
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.05)

    for epoch in tqdm.tqdm(range(EPOCHS)):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            
            m = nn.LogSoftmax(dim = 1)
            outputs = m(net(batch_X))
            loss = loss_function(outputs, batch_y.float())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%100 == 0):
            print(f"Epoch: {epoch}. Loss: {loss}")
            test(net)
            
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm.tqdm(range(len(test_X))):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i]).to(device)  # returns a list, 
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

In [10]:
#Defines and initiates class

REBUILD_DATA = True

class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\New Feature Creation\input_list.npy', 'rb')
    inputs_o = pickle.load(file)

    file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\New Feature Creation\classes_list.npy', 'rb')
    classes_o = pickle.load(file)

    data = zip(inputs_o,classes_o)
    new_inputs = []
    new_classes = []
    for i,j in data:
        if len(i) == 32:
            new_inputs.append(i)
            new_classes.append(j)
            continue
        if len(i) ==28:
            i_new = i + [-1,-1,-1,-1]
            new_inputs.append(i_new)
            new_classes.append(j+[8])
    inputs = new_inputs
    classes = new_classes
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        print(len(inputs))
        for i in range(len(inputs)):
            if len(classes[i]) in [7,8]:
                self.training_data.append([np.array(inputs[i]),np.array(classes[i])])
            else:
                print(f"{len(classes[i])=}")
                excluded += 1
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)
        
if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()

42997


C:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


excluded =  0


In [47]:
#defines nueral net
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
        
net = Net().to(device)
print(net)    

Net(
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=8, bias=True)
)


In [41]:
#define tensors

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
    
Y_w = torch.tensor([i for i in Y_w])

#trying with full place data --- REMOVE --- 
#Y_w = Y
#Traning / Val split    
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y_w[:-val_size]

test_X = X[-val_size:]
test_y = Y_w[-val_size:]
print(len(train_X), len(test_X))

test_X = test_X.to(device)
test_y = test_y.to(device)
    
a = test_X[1]
train_y

4299
38698 4299


tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], dtype=torch.float64)

In [15]:
#indexed win only
Y_wi = []
for i in Y:
    #print(torch.argmin(i))
    Y_wi.append(torch.argmin(i))
Y_wi = torch.tensor(Y_wi)
print(Y_wi)

tensor([1, 1, 3,  ..., 2, 2, 5])


In [11]:
indexed_win = pd.Series(Y_wi)
indexed_win.value_counts()

5    6350
6    6215
4    5916
3    5609
2    5278
1    5047
0    4428
7    4154
dtype: int64

In [ ]:
#Generates places not used currently
Y_p = []
for i in Y:
    place = [x*x for x in i]
    Y_p.append(place)
    
Y_p[1]

In [31]:
#validating model is new
for (i,j) in net.named_parameters():
    print(i,j)

fc1.weight Parameter containing:
tensor([[-5.6785e-02, -1.2779e-01,  5.8464e-02,  ...,  7.2280e-02,
          1.0995e-01, -1.4799e-01],
        [ 2.4413e-02, -5.3036e-02,  3.3540e-02,  ..., -1.6795e-01,
         -1.3196e-01,  3.6532e-02],
        [-5.9726e-02,  2.0770e-02, -2.9291e-03,  ...,  1.1636e-04,
         -1.3056e-01,  1.5016e-01],
        ...,
        [-2.4110e-02, -1.1390e-01,  1.6284e-01,  ...,  2.8663e-02,
          9.8926e-02,  9.6535e-03],
        [-3.3776e-02, -7.4519e-02,  1.4251e-01,  ...,  4.2525e-02,
         -1.6410e-01,  1.7320e-01],
        [ 2.1226e-02,  1.7521e-01, -1.0713e-01,  ..., -1.6778e-01,
          5.4720e-02, -4.3572e-02]], device='cuda:0', requires_grad=True)
fc1.bias Parameter containing:
tensor([ 0.0037,  0.0603, -0.1189,  0.1717, -0.0673, -0.0818,  0.0930, -0.1562,
         0.1468, -0.1413, -0.0032, -0.0045, -0.1678,  0.0129,  0.0450,  0.0566,
        -0.0420,  0.0875, -0.0513, -0.0631,  0.0399, -0.0156, -0.1430,  0.1304,
         0.1744, -0.0735,  

In [46]:
#utalized Win only class, not usefull for ranking placement

def train(net):
    BATCH_SIZE = 10
    EPOCHS = 50
    optimizer = optim.Adam(net.parameters(), lr=0.0001)
    loss_func = nn.MSELoss()
    for epoch in tqdm.tqdm(range(EPOCHS)):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            #print(batch_X, batch_y)

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            optimizer.zero_grad()
            m = nn.LogSoftmax(dim = 1)
            outputs = net(batch_X)
            loss = loss_func(outputs, batch_y.float())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%10 == 0):
            print(f"Epoch: {epoch}. Loss: {loss}")
            correct = 0
            total = 0
            with torch.no_grad():
                for i in tqdm.tqdm(range(len(test_X))):
                    real_class = torch.argmax(test_y[i])
                    net_out = net(test_X)[i] # returns a list, 
                    predicted_class = torch.argmax(net_out)
                    if predicted_class == real_class:
                        idx = int(predicted_class)*4-3
                        #print(predicted_class,net_out, test_X[i][idx])
                        correct += 1

                    total += 1
            print("Accuracy: ", round(correct/total, 3))
            print(f"{correct=}")

In [48]:
train(net)

  0%|          | 0/50 [00:00<?, ?it/s]


Epoch: 0. Loss: 4.96676778793335


  2%|▏         | 1/50 [00:06<05:13,  6.40s/it]

Accuracy:  0.159
correct=684


 20%|██        | 10/50 [00:53<03:31,  5.28s/it]

...

 80%|████████  | 40/50 [03:30<00:50,  5.02s/it]


Epoch: 40. Loss: 4.966092586517334


 82%|████████▏ | 41/50 [03:36<00:48,  5.42s/it]

Accuracy:  0.166
correct=715


 84%|████████▍ | 42/50 [03:41<00:42,  5.29s/it]

In [27]:

correct = 0
total = 0
with torch.no_grad():
    for i in tqdm.tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X)[i] # returns a list, 
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            idx = int(predicted_class)*4-3
            #print(predicted_class,net_out, test_X[i][idx])
            correct += 1

        total += 1
print("Accuracy: ", round(correct/total, 3))
print(f"{correct=}")

100%|██████████| 4299/4299 [00:01<00:00, 2871.10it/s]

Accuracy:  0.171
correct=734


In [26]:
cl = [int(x) for x in correct_list]
cl_series = pd.Series(cl)
wl = [int(x) for x in wrong_list]
wl_series = pd.Series(wl)
plt.hist(cl, color = 'g', alpha=0.5)

#plt.hist(wl, color = 'r', alpha=0.5)
names_c = cl_series.value_counts().index
values_c = cl_series.value_counts().values

names_w = wl_series.value_counts().index
values_w = wl_series.value_counts().values
fig, axs = plt.subplots(1, 2, figsize=(9, 3), sharey=True)

axs[0].bar(names_c,values_c, color = 'g', alpha=0.5)

  0%|          | 0/4299 [00:00<?, ?it/s]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
def test(net):
    correct = 0
    total = 0
    realclasslist = []
    predictedClassList = []
    with torch.no_grad():
        for i in range(len(test_X)):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            realclasslist.append(test_y[i])
            net_out = net(test_X[i]).to(device)  # returns a list, 
            predictedClassList.append(net_out.tolist())
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))
    return (predictedClassList, realclasslist)

In [ ]:
#Test for indexed win only


def test(net):
    correct = 0
    total = 0
    realclasslist = []
    predictedClassList = []
    with torch.no_grad():
        for i in range(len(test_X)):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = test_y[i]
            #print(real_class)
            realclasslist.append(test_y[i])
            net_out = net(test_X[i]).to(device)  # returns a list, 
            predictedClassList.append(net_out.tolist())
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))
    return (predictedClassList, realclasslist)

TypeError: forward() missing 1 required positional argument: 'x'

In [ ]:
winners = [i.item() for i in Y_wi]
from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
    
List = winners
print(most_frequent(List))

4


In [ ]:
#TESTing
m = nn.LogSoftmax(dim = 1)

In [ ]:
for i in range(1000):
    print(torch.argmax(net(test_X)[i]))

tensor(6, device='cuda:0')
tensor(5, device='cuda:0')
tensor(6, device='cuda:0')
tensor(5, device='cuda:0')
tensor(4, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(5, device='cuda:0')
tensor(6, device='cuda:0')
tensor(4, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(5, device='cuda:0')
tensor(6, device='cuda:0')
tensor(7, device='cuda:0')
tensor(6, device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')
tensor(6, device='cuda:0')
tensor(5, device='cuda:0')
tensor(1, device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(7, device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')
tensor(6, device='cuda:0')
tensor(7, device='cuda:0')
tensor(6, device='cuda:0')
t

In [ ]:
from torchvision import transforms, datasets

test = datasets.MNIST('', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

  6%|▋         | 626688/9912422 [00:00<00:01, 6266602.10it/s]

9913344it [00:01, 6792260.79it/s]                             


Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw



29696it [00:00, 14847306.18it/s]         

Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw



...

Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw



5120it [00:00, ?it/s]                   

Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw

Processing...



C:\Users\Nick\.conda\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)
testset


In [ ]:
for data in testset:
    x,y = data
    print(x,y)
    print(len(x))
    break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.